以下のセルを実行したら、メニューの「Kernel」→「Restart」をクリックしてカーネルを再起動してから続きのセルを実行してください。

In [ ]:
!pip install -U pip
!pip install -q -U sagemaker ipywidgets
!pip install --upgrade --no-cache-dir torchvision torch=="1.11.0+cu102"
!pip install timm==0.4.9

# PyTorch EfficientNet-V2 を SageMaker 非同期エンドポイントにデプロイ

このノートブックでは、学習済みの PyTorch の EfficientNet-V2 モデルを SageMaker 非同期推論エンドポイントにデプロイし、エンドポイントの AutoScaling 設定をします。AutoScaling 設定で最小インスタンス数をゼロに設定することにより、推論リクエストがない場合はエンドポイントで使用するインスタンス数をゼロまで Scale In することができます。

**NOTE**: このノートブックは、ノートブックインスタンスの conda_python_p38 で動作を確認しました。

## 推論エンドポイント作成の流れ

まず Model を作成し、Model を使って Endpoint Config を作成します。Endpoint Config を使って Endpoint を起動します。Endpoint には AutoScaling 設定をアタッチすることができます。いったん起動した Endpoint の中の Endpoint Config はダウンタイムなしで更新することが可能です。このしくみによって、エンドポイント起動後にデプロイしたモデルをダウンタイムなしで入れかえることが可能です。

<img src="structure.png" width="70%">

## 使用するライブラリの Import

In [ ]:
import sagemaker
from sagemaker import Session, get_execution_role
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.utils import name_from_base

print(sagemaker.__version__)

sess = Session()
bucket = sess.default_bucket()
role = get_execution_role()
endpoints = {}

timm からモデルをダウンロードし、Amazon SageMaker がこのモデルをデプロイするための形式にします。PyTorch を使用する場合、SageMakerは `.tar.gz` フォーマットの単一のアーカイブファイルを期待しているため、モデルファイルをルートフォルダに、推論用のコードを `code` フォルダに格納します。アーカイブの構造は以下のようになります。


```
/model.tar.gz
/--- model.pth
/--- code/
/--- /--- inference.py
/--- /--- requirements.txt (optional)
```

In [ ]:
import torch
import timm
import tarfile

# Load the model
model = timm.create_model("tf_efficientnetv2_b0", pretrained=True)
model.eval()

input_shape = torch.rand(1, 3, 224, 224)
model_trace = torch.jit.trace(model, input_shape)
model_trace.save('model.pth')

with tarfile.open('gpu_model.tar.gz', 'w:gz') as f:
    f.add('model.pth')
    f.add('code/gpu-inference.py', 'code/inference.py')
f.close()

pytorch_efficientnetv2_prefix = 'pytorch/efficientnetv2'
gpu_model_data = sess.upload_data('gpu_model.tar.gz', bucket, pytorch_efficientnetv2_prefix)    
   
print(f'Model stored in {gpu_model_data}')

## Deploy and test on GPU (ml.g4dn.xlarge)

The instance chosen this time is a `ml.g4dn.xlarge`. It has great throughput and the cheapest way of running GPU inferences on the AWS cloud.

In [ ]:
pth_model = PyTorchModel(model_data=gpu_model_data,
     entry_point='gpu-inference.py',
     source_dir='code',
     role=role,
     framework_version='1.10',
     py_version='py38'
)

In [ ]:
predictor = pth_model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')

endpoints['g4dn'] = predictor.endpoint_name
predictor.endpoint_name

## コンテナイメージの取得

エンドポイントで使用するコンテナイメージを取得します。このサンプルノートブックでは、SageMaker が用意した PyTorch のコンテナイメージを使用します。独自のコンテナイメージを使いたい場合は、[こちらのドキュメント](https://docs.aws.amazon.com/sagemaker/latest/dg/adapt-inference-container.html) や [こちらのサンプルコード](https://github.com/aws/amazon-sagemaker-examples/tree/main/advanced_functionality/multi_model_bring_your_own) を参照してください。

In [ ]:
import boto3
import time
from time import strftime,gmtime
from sagemaker.image_uris import retrieve
boto_session = boto3.session.Session()
sm_client = boto_session.client("sagemaker")
region = boto_session.region_name
sm_runtime = boto_session.client("sagemaker-runtime")

pytorch_inference_image_uri = retrieve('pytorch',
                                       region,
                                       version='1.10',
                                       py_version='py38',
                                       instance_type = 'ml.g4dn.xlarge',
                                       accelerator_type=None,
                                       image_scope='inference')

## Model の作成

In [ ]:
model_name = 'sagemaker-efficientnetv2-{0}'.format(str(int(time.time())))

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': pytorch_inference_image_uri,
        'ModelDataUrl': gpu_model_data,
        'Environment': {
            'TS_MAX_REQUEST_SIZE': '100000000', #default max request size is 6 Mb for torchserve, need to update it to support the 70 mb input payload
            'TS_MAX_RESPONSE_SIZE': '100000000',
            'TS_DEFAULT_RESPONSE_TIMEOUT': '1000'
        }
    },    
)

## Endpoint Config の作成

In [ ]:
endpoint_config_name = f"PyTorchAsyncEndpointConfig-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
bucket_prefix = "async-result"
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1
        }
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            "S3OutputPath": f"s3://{bucket}/{bucket_prefix}/output",
            #  Optionally specify Amazon SNS topics
            "NotificationConfig": {
#               "SuccessTopic": success_topic,
#               "ErrorTopic": error_topic,
            }
        },
        "ClientConfig": {
            "MaxConcurrentInvocationsPerInstance": 2
        }
    }
)
print(f"Created EndpointConfig: {create_endpoint_config_response['EndpointConfigArn']}")

## Endpoint の作成

以下のセルを実行して SageMaker 非同期推論エンドポイントを起動します。エンドポイントが InService になるまで数分かかるので待ちます。SageMaker コンソールの左側のメニューから「推論」→「エンドポイント」とクリックすると、今起動中のエンドポイントのステータスが `Creating` になっているはずです。ここが `InService` になれば推論を開始することができます。

In [ ]:
endpoint_name = f"sm-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
create_endpoint_response = sm_client.create_endpoint(EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name)
print(f"Creating Endpoint: {create_endpoint_response['EndpointArn']}")

## AutoScaling の有効化

エンドポイントのステータスが `InService` になったら、以下のセルを実行してエンドポイントの AutoScaling の設定をします。このときに `MinCapacity` にゼロを指定することで、推論リクエストが発生していない時にインスタンス数をゼロまで Scale In することができます。現在のところ、SageMaker の推論エンドポイントの機能でインスタンス数をゼロまで Scale In できるのは、非同期推論エンドポイントのみです（リアルタイム推論エンドポイントは最小インスタンス数が 1）。

In [ ]:
client = boto3.client('application-autoscaling') # Common class representing Application Auto Scaling for SageMaker amongst other services

resource_id='endpoint/' + endpoint_name + '/variant/' + 'variant1' # This is the format in which application autoscaling references the endpoint

response = client.register_scalable_target(
    ServiceNamespace='sagemaker', 
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    MinCapacity=0,  
    MaxCapacity=5
)

response = client.put_scaling_policy(
    PolicyName='Invocations-ScalingPolicy',
    ServiceNamespace='sagemaker', # The namespace of the AWS service that provides the resource. 
    ResourceId=resource_id, # Endpoint name 
    ScalableDimension='sagemaker:variant:DesiredInstanceCount', # SageMaker supports only Instance Count
    PolicyType='TargetTrackingScaling', # 'StepScaling'|'TargetTrackingScaling'
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 3.0, # The target value for the metric. 
        'CustomizedMetricSpecification': {
            'MetricName': 'ApproximateBacklogSizePerInstance',
            'Namespace': 'AWS/SageMaker',
            'Dimensions': [
                {'Name': 'EndpointName', 'Value': endpoint_name }
            ],
            'Statistic': 'Average',
        },
        'ScaleInCooldown': 20, # The cooldown period helps you prevent your Auto Scaling group from launching or terminating 
                                # additional instances before the effects of previous activities are visible. 
                                # You can configure the length of time based on your instance startup time or other application needs.
                                # ScaleInCooldown - The amount of time, in seconds, after a scale in activity completes before another scale in activity can start. 
        'ScaleOutCooldown': 120 # ScaleOutCooldown - The amount of time, in seconds, after a scale out activity completes before another scale out activity can start.
        
        # 'DisableScaleIn': True|False - ndicates whether scale in by the target tracking policy is disabled. 
                            # If the value is true , scale in is disabled and the target tracking policy won't remove capacity from the scalable resource.
    }
)

## 非同期推論の実行

以下の画像を使って非同期推論を実行してみましょう。非同期推論は推論の入力データが S3 に保存されていることを期待しているため、画像を S3 にアップロードします。

In [ ]:
from IPython.display import Image

Image('cat.jpg')

In [ ]:
image_s3_path = sess.upload_data('cat.jpg', bucket, 'image')
image_s3_path

`invoke_endpoint_async` API で非同期推論を実行します。response の中には、推論結果そのものではなく、推論結果が記載されたファイルの S3 パスの情報が格納されておいます。バックログが溜まっていなければすぐにこのファイルは作成されますが、バックログにリクエストが溜まっている場合は順次推論が実行されるためファイルが作成されるまでに時間がかかることがあります。

In [ ]:
response = sm_runtime.invoke_endpoint_async(
    EndpointName=endpoint_name, 
    InputLocation=image_s3_path)
output_location = response['OutputLocation']
print(f"OutputLocation: {output_location}")

In [ ]:
!aws s3 cp s3://deep-learning-models/image-models/imagenet_class_index.json ./
    
import json
with open("./imagenet_class_index.json", "r") as read_file:
    class_idx = json.load(read_file)

推論結果のファイルを読み込んで確認します。beagle と表示されれば正しい推論結果が得られたと言えるでしょう。

In [ ]:
import urllib
import ast
import numpy as np

output_url = urllib.parse.urlparse(output_location)
output = sess.read_s3_file(bucket=output_url.netloc, key_prefix=output_url.path[1:])
result = ast.literal_eval(output)[0]
pred = np.argmax(result)
class_idx[str(pred)][1]

以下のコマンドを実行すると、エンドポイントが使用しているインスタンス数を `CurrentInstanceCount` として確認することができます。

In [ ]:
response = sm_client.describe_endpoint(
    EndpointName=endpoint_name
)
response['ProductionVariants']

## エンドポイントの更新

現在の SageMaker 非同期エンドポイントでは、AutoScaling によっていったんインスタンス数がゼロになった場合、推論リクエストがバックログに 4つ以上貯まらないと再度インスタンス数の Scale Out が実行されません。そのため、推論リクエストが 1件でも発生したらインスタンスを起動するためのワークアラウンドを紹介します。


### エンドポイントの数を更新

以下のように、エンドポイントに対して `DesiredInstanceCount` を設定し直すことで指定された台数のインスタンスが起動します。以下のセルを実行すると推論絵dのポイントが Updating の状態になり、このサンプルコードの例だと 4分ほどでインスタンスが起動してバックログに積まれた推論リクエストが実行されます。

実際のワークフローでは、上記 `describe_endpoint` API を実行して `CurrentInstanceCount` を確認し、インスタンス数がゼロだったら以下のように `update_endpoint_weights_and_capacities` API を実行してインスタンスを 1つ以上起動するなどの使い方が可能です。

In [ ]:
response = sm_client.update_endpoint_weights_and_capacities(
    EndpointName=endpoint_name,
    DesiredWeightsAndCapacities=[
        {
            'VariantName': 'variant1',
            'DesiredInstanceCount': 1
        },
    ]
)

エンドポイントに関するその他の設定を更新する場合は、エンドポイント設定をパラメタに指定して `update_endpoint` API を実行します。この操作でもいったんゼロになったインスタンス数が `InitialInstanceCount` で指定した値で再度起動されます。

In [ ]:
response = sm_client.update_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

# Clean-up

以下のセルを実行して、不要になったエンドポイントを削除してください。

In [ ]:
pred = sagemaker.predictor.Predictor(endpoint_name=endpoint_name
pred.delete_endpoint()